# Process Mining with sberpm

__`sberpm`__ library is designed to analyse and investigate processes, display them as graphs and solve related classification and clustering problems using machine learning algorithms.

To get the full version of the library, write an email with a request to aabugaenko@sberbank.ru

The page of the SberPM platform:
https://developers.sber.ru/portal/products/sber-process-mining

## Contents

[DataHolder](#DataHolder)

[Synthetic process IDs](#Synthetic-process-IDs) # only full library version

[----------  Traditional Process Mining  ----------](#----------Traditional-Process-Mining----------)

I. [Miners and graph visualisation](#I.-Miners-and-graph-visualisation)
 1. [SimpleMiner](#1.-SimpleMiner)
 2. [CausalMiner](#2.-CausalMiner)
 3. [HeuMiner](#3.-HeuMiner)
 4. [AlphaMiner](#4.-AlphaMiner)
 5. [AlphaPlusMiner](#5.-AlphaPlusMiner)
 6. [InductiveMiner](#6.-InductiveMiner)

II. [Metrics](#II.-Metrics)
- [Metrics + graphs](#Metrics-+-graphs)

III. [Conformance Checking](#III.-Conformance-Checking)

 IV. [BPMN](#IV.-BPMN)

V. [Visualisation](#V.-Visualisation)



[---------- Machine Learning ----------](#----------Machine-Learning----------)

I. [Clustering of stages](#I.-Clustering-of-stages) # only full library version

II. [Automatic search for inefficiencies](#II.-Automatic-search-for-inefficiencies) # only full library version

III. [Anomaly Detection](#III.-Anomaly-Detection) 

IV. [Factor analysis](#IV.-Factor-analysis) # only full library version

V. [Recommender system](#V.-Recommender-system) # only full library version

VI. [Text Analysis](#VI.-Text-Analysis) # only full library version

VII. [Sentimental Analysis](#VII.-Sentimental-Analysis) # only full library version

VIII. [Searching for a Happy Path](#VIII.-Searching-for-a-Happy-Path) # only full library version

IX. [Predicting graph structure](#IX.-Predicting-graph-structure) # only full library version

X. [Simulation modeling and what-if analysis](#X.-Simulation-modeling-and-what-if-analysis)

XI. [Decision Mining](#XI.-Decision-Mining)

XII. [Timing](#XII.-Timing)


# DataHolder

The `DataHolder` is a base class for storing data. Almost all library algorithms work with it (take it as input).

To create a `DataHolder` class, you must first specify file path or pass DataFrame to the constructor and then specify __id_column__ and __activity_column__. However, for most of the Process Mining algorithms in the library, these columns are not sufficient - at least one time column (__start_timestamp_column__ and/or __end_timestamp_column__) and a user column (__user_column__) are required. 

## DataHolder parameters
- **data (str or pd.DataFrame)** – data file path (.csv, .xls(x), .txt) or pd.DataFrame
- **id_column (str)** – id column
- **activity_column (str)** – activity column
- __<font color='red'>*</font>start_timestamp_column (str)__ – start time of activities
- __<font color='red'>*</font>end_timestamp_column (str)__ – end time of activities
- __user_column (str)__ – column with user names/id
- __text_column (str)__ – column with text data
- __duration_column (str)__ – column with activity durations (if not specified, it is calculated as activity_time_2 - activity_time_1, and if there is only one column with time, NaN is set for the last activity in the chain)
- __duration_unit (str)__ – dimension (unit of measure) of the values in the duration_column, if specified

- __sep (str, default=',')__ – delimiter character (only used when reading data from a file)
- __encoding (str)__ – encoding (only used when reading data from a file)
- __nrows (int)__ – number of lines to read (only used when reading data from a file)

- __preprocess (bool, default=True)__ – data preprocessing (sorting, removal of non-values, type conversions)
- __time_format (str)__ – time column format (must be set for correct date recognition and to speed up operation). Formats can be found here: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
- __time_errors: (str, default='raise')__ – action on conversion error
- __dayfirst: (bool, default=None)__ – True, if starts from the day
- __yearfirst: (bool, default=None)__ – True, if starts from the year
- __n_jobs (int, default=1)__ – the maximum number of threads available


__<font color='red'>*</font>__ For most algorithms at least one of the timestamp columns must be specified. If there is no information about the type of column (start or end time), it should be specified as __start_timestamp_column__. For correct format recognition it is also necessary to specify __time_format__.

Data under study should be like an event log (log file) that stores information about the sequence (chain) of events (activities) in the business processes. An example of an event log: $W = \{(a,b,c,d), (a,c,b,d), (a,e,d)\}$ where events $a$, $b$, $c$, $d$ and $e$ are sorted by time.

## DataHolder creation 
### – with DataFrame

In [ ]:
from sberpm import DataHolder
import pandas as pd

df = pd.DataFrame({'id_column': [1, 1, 2, 2, 3, 3],
                   'activity_column': ['st1', 'st2', 'st1', 'st3', 'st1','st2'],
                   'start_timestamp_column': ['10.05.2020', '10.09.2020', '10.03.2020', '10.04.2020', '10.05.2020', '10.05.2020']})

data_holder = DataHolder(data=df, 
                         id_column='id_column', 
                         activity_column='activity_column', 
                         start_timestamp_column='start_timestamp_column', 
                         time_format='%d.%m.%Y')

### – with data file path

In [ ]:
path = 'example.xlsx'
data_holder = DataHolder(data=path, 
                         id_column='id', 
                         activity_column='stages', 
                         start_timestamp_column='dt', 
                         user_column='users', 
                         text_column="some_text",
                         time_format='%Y-%m-%d')

If dataset has a separator of some kind, such as '|' as in csv format, then after setting the columns, need to set the parameter __sep='|'__.

## DataHolder attributes
In the `DataHolder` the column names are stored in the corresponding variables (i.e. there is no need to remember the column names):
- id_column
- activity_column
- start_timestamp_column
- end_timestamp_column
- user_column
- text_column
- duration_column

In addition, the `DataHolder` stores raw and grouped data as a DataFrame, which can be accessed as follows
- data
- grouped_data

## DataHolder methods
- __check_or_calc_duration__ – calculates the duration of each activity (in seconds) if necessary.
- __get_grouped_data__ – outputs grouped data by id and specified columns (e.g, by activity_column and by start_timestamp_column)
- __get_unique_activities__ – displays a list of unique activities
- __get_columns__ – displays a list with column names 
- __get_text__ – displays the text column, if there is one
- __get_timestamp_col__ – outputs a temporary column; if there are 2, outputs start_time_column
- __is_interval__ – returns True if it is an "interval log" (which has both time columns: start and end of activity)
- __top_traces_dh__ – returns the data_holder with data for the n most frequent chains

In [ ]:
data_holder.check_or_calc_duration()

In [ ]:
data_holder.data.head()

In [ ]:
data_holder.get_grouped_data(data_holder.activity_column, data_holder.start_timestamp_column).head()

In [ ]:
dh_3 = data_holder.top_traces_dh(3)  # data for the top 3 chains only
dfg = dh_3.get_grouped_data(dh_3.activity_column)
dfg.value_counts(dh_3.activity_column)  # check

Once you have business process data with status chains and start times for each of them, you can load them into the `DataHolder` and build a graph that describes this business process as much as possible.

# Synthetic process IDs

##### <font color='red'>Full library version</font>

__Pro_n_check__ Numerates the process instances according to the given conditions. Creates a 'pro_n' column.

# ----------Traditional Process Mining----------

## I. Miners and graph visualisation

Several algorithms are implemented in the library to build and draw the process graph. All of them are stored in the __`sberpm.miners`__ module and have one method:
- __apply__ - builds graph, which is saved in the graph field.

### 1. SimpleMiner

`SimpleMiner` draws all edges found in the log (without any filtering).

In terms of Process Mining:
> If in at least one chain of activities from the log, some activity $X$ is directly followed by an activity $Y$ (a chain of the form $...XY...$), then write $X>Y$ ($Y$ follows $X$, _follows_ relation).

SimpleMiner draws edges between such pairs of activities $X$ and $Y$ if $X>Y$ is true.

In [ ]:
from sberpm.miners import SimpleMiner

In [ ]:
# Creating a SimpleMiner object. The DataHolder and algorithm parameters are pass to the constructor 
# (this miner has no parameters)
simple_miner = SimpleMiner(data_holder)

# Start the graph algorithm
simple_miner.apply()

# Saving a graph
graph = simple_miner.graph

### Graph visualization
To visualise a graph, use `GraphvizPainter` from module __`sberpm.visual`__

The class `GraphvizPainter` has methods:
- __apply__ - accepts the graph from the miner and makes calculation for rendering it 
- __write_graph__ - saves the graph in the required format (pdf, svg, gv, png)
- __show__ - displays the graph in notebook

In [ ]:
# Creating a GraphvizPainter object
painter = GraphvizPainter()

# Calculating the graph from the SimpleMiner results
painter.apply(graph)

# You can save the graph to your hard disk in png, svg, pdf or gv format
painter.write_graph('SimpleMiner.png', format='png')

# You can display the graph in notebook
painter.show()

The class `Graph` from module __`sberpm.visual`__ has methods:
- __get_nodes__ - get all nodes
- __get_edges__ - get all edges
- __add_node_metric__ - add metric related to graph nodes
- __add_edge_metric__ - add metric related to graph edges
- __clear_node_metrics__ - delete all metrics from nodes
- __clear_edge_metrics__ - remove all metrics from edges

### 2. CausalMiner

`CausalMiner` is based on filtering of edges.
> Derived types of relations from $X>Y$:
- direct relations ($X \to Y$, _causal_ relation) are relations where $X>Y$ and not $Y>X$
- parallel relations ($X\parallel Y$, _parallell_ relation) are relations where $X>Y$ and $Y>X$
- independent relations ($X\#Y$, independent) are relations where neither $X>Y$ nor $Y>X$

The CausalMiner draws edges between such pairs of activities $X$ and $Y$ if $X\to Y$ is true.

In [ ]:
from sberpm.miners import CausalMiner

In [ ]:
# Miner
causal_miner = CausalMiner(data_holder)
causal_miner.apply()
graph = causal_miner.graph

# Displaying graph
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 3. HeuMiner

`HeuMiner` – is a heuristic miner that removes the rarest connections depending on the threshold set. 

The **threshold** parameter takes values **from 0 to 1**. The higher it is, the fewer edges in the graph (the remaining edges are considered more important).

Source: https://www.researchgate.net/publication/229124308_Process_Mining_with_the_Heuristics_Miner-algorithm

In [ ]:
from sberpm.miners import HeuMiner

In [ ]:
# Miner
heu_miner = HeuMiner(data_holder, threshold=0.8)
heu_miner.apply()
graph = heu_miner.graph

# Displaying graph
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 4. AlphaMiner

`AlphaMiner` draws a graph in the form of Petri net, taking into account direct, parallel and independent relations between activities. 

In [ ]:
from sberpm.miners import AlphaMiner

In [ ]:
# Miner
alpha_miner = AlphaMiner(data_holder)
alpha_miner.apply()
graph = alpha_miner.graph

# Displaying graph
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 5. AlphaPlusMiner

`AlphaPlusMiner` – implementation of Alpha+ Miner, which also draws a graph in the form of Petri net with relations, but unlike AlphaMiner can work with one-loop chains of the form activity_1$\to$activity_1 (self-loop).

In [ ]:
from sberpm.miners import AlphaPlusMiner

In [ ]:
# Miner
alpha_miner_plus = AlphaPlusMiner(data_holder)
alpha_miner_plus.apply()
graph = alpha_miner_plus.graph

# Displaying graph
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 6. InductiveMiner

`InductiveMiner` creates a process tree. The foxes of the tree are the actual process activities, the other nodes are the operators. There are 4 types of operators: 
- SEQUENTIAL (`->`), 
- EXCLUSIVE OR (`X`), 
- PARALLEL (`||`), 
- CYCLE (`*`).

There is an additional 'operator' that says that it was not possible to find any of the 4 operators above:
- MIXED MODEL ('`?)

*Note*: some of the tree leaves may be *hidden activities*, displayed as black rectangles. They are not real activities and are only used to preserve the correct tree structure. 

For example, from a log of two process chains $W = \{(a, b, c), (a, c)\}$, you can get the following process tree:        
`->(a, X(b, hidden_activity), c)`.

If during the next iteration the algorithm cannot find the graph slice (=select one of the 4 operators), the following behaviour can be added: if there exists activity A, when removing which operator can be found, the algorithm returns the following tree:            
`||(X(activity_A, hidden_activity), graph_without_activity_A)` - |activity A is considered parallel to the rest of the graph.


This behavior can be turned on or off with parameter **parallell_activity** in class `InductiveMiner`.

In [ ]:
from sberpm.miners import InductiveMiner

In [ ]:
# Miner
inductive_miner = InductiveMiner(data_holder)
inductive_miner.apply()
graph = inductive_miner.graph

# Displaying graph
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

## II. Metrics

There are currently 5 main types of metrics in the __`sberpm.metrics`__ module:
1. `ActivityMetric` - metrics for activity (grouping by activity_column)
2. `TransitionMetric` - metrics for transitions (grouping by unique transitions)
3. `IdMetric` - metrics for id (grouping by id_column)
4. `TraceMetric` - metrics for chains of activities (grouping by unique strings)
5. `UserMetric` - metrics for users (grouping by user_column)

In [ ]:
from sberpm.metrics import ActivityMetric, TransitionMetric, IdMetric, TraceMetric, UserMetric

Parameters:
- __data_holder__ - object of type DataHolder for which metrics should be calculated
- __time_unit__ - time unit, by default time metrics are calculated in hours
- __round__ - number of digits after decimal point (only for metrics that can be floating-point)

Methods common to all classes:
- __apply__ - calculation of all characteristics
- __calc_metrics(...)__ - calculate the specified metrics (corresponds to methods/column names in DataFrame from apply)
- __calculate_time_metrics__ - calculation of time characteristics

- __total_duration__ - total operating time
- __min_duration__ - minimum operating time
- __max_duration__ - maximum operating time
- __mean_duration__ - mean operating time
- __median_duration__ - median operating time
- __std_duration__ - standard deviation of operating time
- __var_duration__ - operating time variance

Additional methods:
- ActivityMetric
    - __count__ - number of times the activity occurs in the log
    - __unique_ids__ - unique id for each activity
    - __unique_ids_num__ - number of unique id for each activity
    - __aver_count_in_trace__ - average number of times an activity occurs in a chain
    - __loop_percent__ - percentage of looping
    - __throughput__ - frequency - number of activities performed per unit time
    - __unique_users__ - unique users who performed the activity
    - __unique_users_num__ - number of unique users working on the activity
    - __success_rate(...)__ - share of id's having the given activity and succeded
    - __failure_rate(...)__ - share of id's, having the given activity and failed (ended with unsuccessful activities)
    
    
- IdMetric
    - __trace__ - chain (list of activities)
    - __trace_length__ - chain length (number of activities in the chain)
    - __unique_activities__ - unique activities in a chain
    - __unique_activities_num__ - number of unique activities in a chain
    - __loop_percent__ - looping percentage
    - __unique_users__ - unique users with this ID
    - __unique_users_num__ - number of unique users working with this ID


- TraceMetric
    - __count__ - how many times this chain occurs in the log file
    - __ids__ - unique id with the given chain
    - __trace_length__ - the length of the chain
    - __unique_activities__ - unique activities in the chain
    - __unique_activities_num__ - number of unique activities in the chain
    - __unique_users__ - unique users working on the chain of activities
    - __unique_users_num__ - number of unique users working on the chain of activities

 
- TransitionMetric
    - __count__ - number of times the transition occurs in the log file
    - __unique_ids__ - unique id for each transition
    - __unique_ids_num__ - number of unique ids for each transition
    - __aver_count_in_trace__ - average number of times the object occurs in the chain
    - __loop_percent__ - percentage of looping
    - __throughput__ - number of transitions per time unit
    - __unique_users__ - unique users working on the object
    - __unique_users_num__ - number of unique users working on the object
    - __success_rate(...)__ - share of id's having given transition and succeded
    - __failure_rate(...)__ - share of id's having given transition and failed.
    
    
- UserMetric
    - __count__ - how many times the given user occurs in the log
    - __unique_activities__ - unique activities that the user was working with
    - __unique_activities_num__ - number of unique activities the user was working with
    - __unique_ids__ - unique id with the user
    - __unique_ids_num__ - number of unique ids with the given user
    - __throughput__ - number of times the object was executed per time unit
    - __workload__ - share of logging activity performed by the given user

### 1. ActivityMetric

In [ ]:
# Creating ActivityMetric object
activity_metric = ActivityMetric(data_holder, time_unit='d')

# Calculation of all metrics
activity_metric.apply().head()

### 2. TransitionMetric

In [ ]:
# Creating TransitionMetric object
transition_metric = TransitionMetric(data_holder, time_unit='d')

# Calculation of all metrics
transition_metric.apply().head()

### 3. IdMetric

In [ ]:
# Creating IdMetric object
id_metric = IdMetric(data_holder, time_unit='d')

# Calculation of all metrics
id_metric.apply().head()

### 4. TraceMetric

In [ ]:
# Creating TraceMetric object
trace_metric = TraceMetric(data_holder, time_unit='d')

# Calculation of all metrics
trace_metric.apply().head()

### 5. UserMetric

In [ ]:
# Creating UserMetric object
user_metric = UserMetric(data_holder, time_unit='d')

# Calculation of all metrics
user_metric.apply().head()

### Metrics + graphs

Library implements the ability to represent some metrics on a graph. It can be done in `Graph` class using methods:
- __add_node_metric__ - add metric associated with nodes of the graph
- __add_edge_metric__ - add metric related to edges of the graph

In [ ]:
# Calculation of metrics
nodes_count_metric = activity_metric.count().to_dict()
edges_count_metric = transition_metric.count().to_dict()
mean_time_node_metric = activity_metric.mean_duration().fillna(0).to_dict()

# Obtaining a graph from a miner
graph = causal_miner.graph

# Adding metrics to the graph
graph.add_node_metric('count', nodes_count_metric)
graph.add_edge_metric('count', edges_count_metric)
graph.add_node_metric('mean_time', mean_time_node_metric)

In [ ]:
# Creating a GraphvizPainter object
painter = GraphvizPainter()

# Draw the graph and relate the colour of the nodes and edges to the required metrics
painter.apply(graph, node_style_metric='count', edge_style_metric='count')
# or painter.apply(graph, node_style_metric='mean_time', edge_style_metric='count')

# Saving graph
painter.write_graph("metric_graph.png", format = 'png')

# Displaying graph
painter.show()

To remove metrics from a graph, use the following methods:
- __clear_node_metrics__ - remove all metrics from nodes
- __clear_edge_metrics__ - remove all metrics from edges

In [ ]:
graph.clear_node_metrics()
graph.clear_edge_metrics()

## III. Conformance Checking

### TokenReplay

`TokenReplay` allows to calculate *fitness*, which shows how well the graph describes the business process (1 - good, 0 - bad). Fitness is calculated separately for each chain (id) when playing it over the Petri net using the following formula:
$$ Fitness = \frac{1}{2}\Big(1-\frac{missed}{consumed}\Big) + \frac{1}{2}\Big(1-\frac{remaining}{produced}\Big) $$
- produced tokens - created as a result of transition
- consumed tokens - removed as a result of the transition
- remaining tokens - remained at the end of the playback
- missing tokens - did not exist, but they are necessary for playback, so they are inserted

The library outputs the following metrics:
- above 4 tokens' metrics and fitness of each chain
- fitness averaged over all chains (__mean_fitness__)
- fitness across the entire log - the sum of above 4 tokens' metrics across all chains in the log (__average_fitness__) is passed to the formula

In [ ]:
from sberpm.conformance_checking import TokenReplay

In [ ]:
token_replay = TokenReplay(data_holder, alpha_miner.graph)
token_replay.apply()
token_replay.result

In [ ]:
print('mean:', token_replay.mean_fitness)
print('average:', token_replay.average_fitness)

Also available in the library is the more general ConformanceChecking class, which holds TokenReplay and a host of other metrics:
- precision
- generalization
- simplicity

In [ ]:
from sberpm.conformance_checking import ConformanceChecking

In [ ]:
cc = ConformanceChecking(data_holder, alpha_miner.graph)
cc.get_conformance_checking()

In [ ]:
cc.get_fitness_df()

## IV. BPMN

To save the graph in BPMN (Business Process Model and Notation) format, you can use `BpmnExporter` from module __`sberpm.bpmn`__. It has the following methods:
- __apply_petri__ - build BPMN for Petri net
- __get_string_representation__ - get BPMN graph notation
- __write__ - write the graph in BPMN format

At the moment only the graphs from Alpha Miner can be saved.

In [ ]:
from sberpm.bpmn import BpmnExporter

In [ ]:
bpmn_exporter = BpmnExporter()
bpmn_exporter.apply(alpha_miner.graph)
bpmn_exporter.get_string_representation()[:1000]

In [ ]:
bpmn_exporter.write('exported.bpmn')

There is a class `BpmnImporter` with the following methods to load BPMN-file:
- __load_bpmn_from_xml__ - load the graph represented as BPMN
- __get_pydotplus_graph__ - get the graph in the pydotplus format

In [ ]:
from sberpm.bpmn import BpmnImporter

In [ ]:
bpmn_importer = BpmnImporter()
bpmn_importer.load_bpmn_from_xml('exported.bpmn')
pydot_graph = bpmn_importer.get_pydotplus_graph()
pydot_graph.write('imported_bpmn.svg', prog='dot', format='svg')

## V. Visualisation

Class `ChartPainter` from module __`berpm.visual`__ is designed to create basic types of charts. The visualisation is based on the __`plotly`__ library, which makes all charts interactive. 

In [ ]:
from sberpm.visual import ChartPainter

Parameters:
- __data__ - data to be visualised (DataFrame, DataHolder or metrics class object)
- __template__ - style of the charts, default is _plotly_
- __palette__ - colour palette of graphs, default _sequential.Sunset_r_

Each `ChartPainter' method allows to draw a graph of a certain type:
- __hist__ - Histogram
- __bar__ - bar graph
- __box__ - box plot
- __scatter__ - scatter plot
- __line__ - line chart
- __pie__ - pie chart
- __sunburst__ - sunburst diagram
- __heatmap__ - 2D histogram
- __timeline__ - Gantt chart
- __pareto__ - Pareto diagram

The main parameters of the methods (see documentation for details):
- __x__, __y__ - names of columns to be drawn on X and Y axes correspondingly
- __sort__ - names of column to sort values
- __n__ - number of rows to be rendered
- __color__ - name of the column to set colour for the chart items
- __subplots__ - tuple of the form (rows, cols, ncols), where rows and cols are the names of the columns to draw multiple graphs by rows and columns respectively, and ncols is the number of columns
- __text__ - name of the column with textual information (or its type) to be shown on the chart
- __orientation__ - graphic orientation
- __opacity__ - transparency of the chart elements
- __edge__ - boundaries of the chart elements
- __title__ - title of the chart

Each method is easy to use, but has a sufficiently wide functionality that allows you to build charts for any task.

### Histogram

In [ ]:
painter = ChartPainter(id_metric)
painter.hist(x='total_duration', edge=True)

### Bar graph

In [ ]:
painter = ChartPainter(user_metric)
painter.bar(x=data_holder.user_column, y='total_duration', text=True)

### Scatter plot

In [ ]:
painter = ChartPainter(id_metric)
painter.scatter(x='mean_duration', y='median_duration', color='unique_users_num', size='trace_length', 
                edge=True, opacity=0.8)

### Pie chart

In [ ]:
painter = ChartPainter(user_metric)
painter.pie(labels='count', n=15)

### Histogram of activity distribution over time ranges

##### For all activities

In [ ]:
painter = ChartPainter(data_holder)
painter.hist_activity_of_dur(top= False, use_median=False)

##### For top activities

In [ ]:
painter.hist_activity_of_dur(top= True)

##### Per activity

In [ ]:
painter.hist_activity_of_dur(by_activity='Stage_6')

# ---------- Machine Learning ----------

## I. Clustering of stages

##### <font color='red'>Full library version</font>

This module is used to cluster the process steps, to find close or identical process steps. 

##  II. Automatic search for inefficiencies

##### <font color='red'>Full library version</font>

The automatic inefficiency search module __`sberpm.autoinsights`__ enables the automatic identification of __weaknesses and process vulnerabilities__ and demonstrates them visually in a process graph. The analysis by the `AutoInsights` class takes into account factors such as:
1. The duration of the stage
2. Increase of step duration
3. Stage irregularity (rarity)
4. Stage has a bottleneck with low variation
5. Stage has a bottleneck with high variation
6. Stage has a longer duration due to frequent incidents
7. Stage has a longer duration because of recurring incidents
8. Step increases the process time and/or other steps
9. Step is run with errors, resulting in slowdown of the process
10. Stage is run with critical system errors, which leads to the failure of the process
11. Stage is run with structural errors which lead to failure of the process
12. Storming at this stage leads to process slowdown failure
13. Reversal at this stage leads to process failure
14. Level of abnormality
15. Sum of financial effects

Results of the calculations are two parameters:
- __Anomaly level - *[0, 1]*__.

    For each object (activity and edge), its anomaly level is considered - a metric with values from 0 to 1 inclusive. The higher the anomaly level, the more insights the object can provide.
    
    
- __Sum of financial effects - *[0, +inf)*__
    
    The sum of financial effects is the sum of financial effects of each metric in the table obtained through *get_clustered_result*. The financial effect is calculated for each activity based on the cost per 'second of human work' *sec_cost*. Also, depending on the metric, the financial effect is influenced by the durations of steps, cycles and other problems of the activity or activities on which the current activity depends. 

## III. Anomaly detection

__Anomaly detection__ (also __outlier detection__) is the recognition of rare data, events, or observations that are suspicious because they are significantly different from most of the data.

To search for anomalies (outliers) in the data, the library has a module __`sberpm.ml.anomaly_detection`__, which has classes `OutlierCBLOF`, `OutlierForest`, `OutlierLOF`, `OutlierOCSVM`, `OutlierCustom`, `OutlierEnsemble`. Each class implements its own __Anomaly Detection Without Teacher__ algorithm, which detects anomalies in unlabeled datasets, assuming most of the dataset is normal, by looking for representatives that fit less closely to the rest of the dataset. 

`OutlierEnsemble' is a composition of anomaly detection algorithms whose final answer is a vote (an object is considered an outlier if most algorithms have defined it as an outlier) of the following algorithms: [KNN](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.knn), [ABOD](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.abod), [HBOS](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.hbos), [Isolation Forest](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.iforest).

In [ ]:
from sberpm.ml.anomaly_detection import OutlierCBLOF, OutlierForest, OutlierLOF, \
                                        OutlierOCSVM, OutlierCustom, OutlierEnsemble

As a parameter, the object takes as input DataHolder, for which it calculates basic statistics, such as average time, activity chain length, number of unique users (if any), etc. 

In [ ]:
outlier_detector = OutlierForest(data_holder)

Each class has the following methods:
- __add_feature__ - adding the trait by which you want to find anomalies 
- __add_groupby_feature__ - add a feature for finding anomalies, calculated by grouped data 
- __apply__ - start the algorithm
- __get_outlier_ids__ - displaying the id of the anomalous processes
- __print_result__ - display the statistics on the anomalies
- __show_permutation_importance__ - illustration of permutation importance of features by which outliers differ (works everywhere except `OutlierEnsemble`)

In [ ]:
# Adding a feature named max_time, calculated by applying the max function to the column 
# data_holder.duration_column in id grouped data (maximum activity time in the process)
outlier_detector.add_groupby_feature('max_time', data_holder.duration_column, max)

The module implements 5 anomaly detection techniques:
1. __Isolation Forest (IF)__
2. __One-Class Support Vector Machines (OCSVM)__
3. __Local Outlier Factor (LOF)__
4. __Cluster-Based Local Outlier Factor (CBLOF)__
5. __OutlierEnsemble__ which has __KNN__, __HBOS__, __ABOD__, __Isolation Forest__

You can also use any other anomaly search algorithm (e.g. from __pyod__ library).

### Isolation Forest

> The __isolation forest__ technique is based on the idea that abnormal observations are easier to separate from the rest (normal) objects of the dataset. The algorithm builds an ensemble of isolating binary decision trees, in each node of which a feature and a partitioning threshold are chosen randomly. The tree is constructed until only one object or objects with the same values remain in the list. Intuitively, __anomalous__ points are those that have a shorter path length in the tree, which is defined as the number of edges that the object passes from the root node to the leaf. 

In [ ]:
outlier_detector = OutlierForest(data_holder)

### One-Class Support Vector Machines

> The main idea of the classical __vector reference method (SVM)__ is to separate objects belonging to different classes by a hyperplane so as to maximize the distance between them. The __OCSVM__ algorithm, as the name implies, learns from data belonging to one class, the class of normal objects. It defines the boundaries of these objects and classifies all other points lying on the other side of the dividing surface as __anomalous__.

In [ ]:
outlier_detector = OutlierOCSVM(data_holder)

### Local Outlier Factor

> The __Local Outlier Level (LOF)__ is based on the concept of local density of an object, where locality is given by its $k$ nearest neighbors, whose distances are used as density estimates. By comparing the local density of an object with the local density of its neighbors, we can distinguish regions with similar density and points that have significantly lower density than its neighbors. These points are considered __outliers__.

In [ ]:
outlier_detector = OutlierLOF(data_holder)

### Cluster-Based Local Outlier Factor

> Unlike the standard LOF, which is based on a metric approach to identify local outliers, __CBLOF__ identifies the cluster structure of the data, divides clusters into "large" and "small" and then determines the locality of small clusters with respect to large ones. Clusters whose locality with respect to others is small are defined as __outliers__.

In [ ]:
outlier_detector = OutlierCBLOF(data_holder)

In addition to these 4 algorithms of anomaly search, you can use any other algorithm that is not built into the library, but is available in pyod - for example, __histogram-based outlier detection (HBOS)__.

In [ ]:
from pyod.models.hbos import HBOS

hbos = HBOS(contamination=0.1)
outlier_detector = OutlierCustom(data_holder, hbos, outlier_label=1)

After selecting the algorithm, it should be applied using the __apply__ method.

In [ ]:
outlier_detector.apply()

The results are a list of anomaly id (__get_outlier_ids__ method), a table with descriptive statistics on anomalous and normal objects (__print_result__ method), and a graphical illustration of the importance of the features used to find anomalies (__show_permutation_importance__ method). 

In [ ]:
outlier_detector.get_outlier_ids()

In [ ]:
outlier_detector.print_result()

In [ ]:
outlier_detector.show_permutation_importance()

### `OutlierEnsemble`
> `OutlierEnsemble` is a composition of anomaly detection algorithms, the final answer of which is a vote (an object is considered an outlier if most algorithms have defined it as an outlier) of the following algorithms: [KNN](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.knn), [ABOD](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.abod), [HBOS](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.hbos), [Isolation Forest](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.iforest). Any subset of {"HBOS", "ABOD", "KNN", "IForest"} can be selected as algorithms.

In [ ]:
outlier_detector = OutlierEnsemble(data_holder, ["HBOS", "ABOD", "KNN", "IForest"])

In [ ]:
outlier_detector.apply()

In [ ]:
outlier_detector.print_result()

## IV. Factor analysis

##### <font color='red'>Full library version</font>


__Factor analysis__ is a multivariate method used to examine relationships between values of variables. It is assumed that known variables depend on a smaller number of unknown variables and random error. With the help of factor analysis it is possible to identify hidden factor variables responsible for the presence of linear statistical correlations between the observed variables.


## V. Recommender system

##### <font color='red'>Full library version</font>


This module is an advisory system that ranks the process steps in order of priority for reengineering. The system shows which activities to focus on first when optimizing the process. First, the best model is selected to describe the dependence of the target metric (metric) of each Activity on the attribute metrics (metric_f) of all other Activities. The metrics may be the number of recurrences (the number of activity occurrences, __"appearance"__), execution time (__"time"__), number of repetitions (__"recycles"__), custom user metric ("__user_metric__"). Then, based on the metric, problem coefficients are constructed, in descending order of which the Activity is ranked.

## VI. Text Analysis

##### <font color='red'>Full library version</font>


This module is used to cluster texts. 
For each cluster, the algorithm outputs a __cluster number__ (cluster name or closest message to the cluster centroid), and the 10 most common words in the cluster.

## VII. Sentimental Analysis

##### <font color='red'>Full library version</font>


This module is a system for analyzing the tone of verbal comments in the text box. The tonality analysis module has two modes: "basic" and "advanced". In the "basic" mode, the tone of the text is defined as "positive" or "negative", the numerical value of the tone is defined in the range from -1 to 1 (from negative to positive).

## VIII. Searching for a Happy Path

##### <font color='red'>Full library version</font>


The task of finding a happy path can be solved by reinforcement learning (RL), which is inherently adapted to the search for optimal actions. RL works with two entities: the agent and the environment. During learning, the agent interacts with the environment, performs actions and receives feedback, which is called a reward. The problem is formulated within a Markovian decision-making process, which is based on:
* a set of states in the world 
* a set of actions 
* a probabilistic distribution of the next state subject to the current state and the completed action 
* the reward in the transition between states when actions are performed. 

The fulfillment of the Markov property is that the next state is conditionally independent of past states and actions, given the current state and action. The process graph is treated as an environment, states are nodes of the graph (activities), actions are edges (choosing the next activity to transition), and the reward is the average negative transition time between past and present states. If there are key states, the transition to them is also rewarded. The goal is to choose an optimal policy - a mapping from state space to action space or, in other words, a guide to action in each state - that maximizes the expected discounted amount of rewards passing through the process graph.
The optimal policy and, as a consequence, the path is found with AutoRL using the best discounted reward sum method from: value iteration, Q-learning, cross entropy, genetic algorithm.



## IX. Predicting graph structure

##### <font color='red'>Full library version</font>


The __GSPredictor__ (graph structure predictor) class contains an algorithm for predicting graph structure, namely, two quantities are predicted: probabilities and average times of graph nodes and edges. These values are represented as time series from the available data, then ml- and time series-specific algorithms are used for prediction.

## X. Simulation modeling and what-if analysis

The simulation module __`sberpm.imitation` allows to simulate the process in as-is form, make changes to the process and perform what-if simulations, and evaluate the quality of the simulation compared to the original log file using the `Simulation` and `SimilarityMetric` classes respectively. 

In [ ]:
from sberpm.imitation import Simulation, SimilarityMetric

Methods of the `Simulation` class:
- __run__ - start simulation num_traces (number) of activity chains (id)
- __to_initial_state__ - return process to initial state
- __mean_duration__ - average duration of activities or transitions in the process
- __change_node_duration__ and __change_edge_duration__ - limit the execution time of a specific activity or transition
- __delete_node__ and __delete_edge__ - deleting a node (activity) and an edge (transition) from the process

Class accepts an object of type DataHolder as an input. It is also possible to fix __random_state__ to make results reproducible. 

In [ ]:
# Initializing
sim = Simulation(data_holder) 

### As-is modelling

 __run__ method starts the process simulation. It has parameters __num_traces__ - number of event chains to simulate and __max_trace_length__ - maximum length of the generated chain (by default 100). 

In [ ]:
# Simulation of the initial number of chains (id)
sim_data = sim.run(num_of_traces=len(data_holder.grouped_data))
sim_data.head()

Resulting log can be rendered using the miner and the built-in graph visualization tool.

In [ ]:
# Creating a DataHodler for the generated data
holder_sim = DataHolder(sim_data, 'id', 'stages', 'dt')

# Miner
miner = HeuMiner(holder_sim)
miner.apply()

# Displaying graph
painter = GraphvizPainter()
painter.apply(miner.graph)
painter.show()

### Model validation

Quality of the simulation can be checked using the `SimilarityMetric` class, which measures the similarity of the original and generated log using the _Damerau-Levenstein distance_. The class takes as input the log from the simulation and the DataHolder of the source file. The result is stored in the __similarity__ field.

In [ ]:
%%time

sim_metric = SimilarityMetric(sim_data, data_holder)
print('Сходство:', sim_metric.similarity)

It is also possible to get the similarity metric for each chain from the generated log separately.

In [ ]:
sim_metric.result.head()

Quality of the simulation is poor due to the peculiarities of the synthetic dataset. The quality on the real log file is much higher.

### What-if analysis

Such methods as __delete_node__ and __delete_edge__ allow you to remove an activity or edge in a process. After running the simulation, the process will be implemented using alternative paths. The __node__ and __edge__ parameters are the names of the activity and transition in the process, respectively.

In [ ]:
sim.delete_node('Stage_7')

In [ ]:
# Process simulation without a node 
sim_data = sim.run(num_of_traces=len(data_holder.grouped_data))

# Creating a DataHolder
holder_sim = DataHolder(sim_data, 'id', 'stages', 'dt')

# Miner
miner = HeuMiner(holder_sim)
miner.apply()

# Displaying graph
painter = GraphvizPainter()
painter.apply(miner.graph)
painter.show()

To return the process to its original state, __to_initial_state__ method must be used.

In [ ]:
sim.to_initial_state()

It is possible to use the __mean_duration__ method to analyze the average duration of all activities or transitions in a process (depends on the __mode__ parameter).

In [ ]:
sim.mean_duration(target='activities')

Such methods as __change_activity_duration__ and __change_transition_duration__ allow to limit the execution time of a particular activity or transition. They have the following parameters:
- __activity__ or __transition__ - name of the activity or transition to limit
- __threshold__ - maximum duration of an activity or transition
- __scale__ - coefficient of decreasing (if > 1) or increasing (if < 1) the duration of activity or transition

In [ ]:
sim.change_activity_duration(activity='Stage_0', scale=3)  # reduce by a factor of 3

In [ ]:
# Simulation with reduced node duration
sim_data = sim.run(num_of_traces=len(data_holder.grouped_data))

# Creating a DataHolder
holder_sim = DataHolder(sim_data, 'id', 'stages', 'dt')

# Calculation of metrics to add to the graph
time_metric = ActivityMetric(holder_sim, time_unit='s')
mean_time_node_metric = time_metric.mean_duration().fillna(0).to_dict()

# Miner
miner = HeuMiner(holder_sim)
miner.apply()
graph = miner.graph
graph.add_node_metric('mean_time', mean_time_node_metric)

# Displaying graph
painter = GraphvizPainter()
painter.apply(miner.graph)
painter.show()

In [ ]:
sim.mean_duration('activities')

In [ ]:
# Return to the original state
sim.to_initial_state()

##  XI. Decision Mining

Module __`sberpm.decision_mining`__ is designed to perform __decision point analysis__, which is to determine the reasons why a process goes one way or another. The `DecisionMining` class identifies how certain properties (attributes) of a process affect the choice of a particular path. 

In [ ]:
from sberpm.decision_mining import DecisionMining

As an input `DecisionMining` takes an object of type DataHolder. 

In [ ]:
# Initialization
dm = DecisionMining(data_holder)

`DecisionMining` has the following methods:
- __print_decision_points__ - outputs decision points (activities followed by choices)
- __apply__ - performs analysis of decision points, building decision tree by specified attributes
- __get_clf_metrics__ - displays the classification metrics
- __plot_confusion_matrix__ - draws error matrices
- __plot_feature_importance__ - draws importance of attributes in the tree
- __plot_feature_distribution__ - draws distribution of features by classes 
- __plot_decision_tree__ - draws the decision tree
- __print_decision_rule__ - outputs decision rules

Decision points - the points where the process has a branching can be viewed using the __print_decision_points__ method.

In [ ]:
dm.print_decision_points()

Method __apply__ runs the decision mining algorithm. It has the following parameters:
- __categorical_attrs__ - names of categorical features
- __noncategorical_attrs__ - names of noncategorical features
- __decision_points__ - points to build decision trees on, by default all are considered
- __sampling__ - whether sampling (over- or under-) is needed, should be used in case of unbalanced classes
- __tree_params__ - parameters of the decision tree
- __grid_search__ - whether selection of optimal hyperparameters of the decision tree is needed
- __param_grid__ - parameter grid, only used if grid_search=True
- __random_state__ - used in the decision tree and sampling
- __n_jobs__ - used in sampling and grid_search

In [ ]:
dm.apply(categorical_attrs=[data_holder.user_column],
         noncategorical_attrs=[data_holder.duration_column],
         decision_points='all', 
         sampling='RandomOverSampler',
         tree_params='default',
         grid_search=False, 
         param_grid='default',
         random_state=42,
         n_jobs=None)

Result of classification can be displayed using the __get_clf_metrics__, __plot_confusion_matrix__ and __plot_feature_importance__ methods. 

In [ ]:
dm.get_clf_metrics()

The quality is poor due to the peculiarities of the synthetic dataset.

All plot methods have parameters:
- __decision_points__ - the points for which you want to plot the charateristics
- __savefig__ - whether to save the image

In [ ]:
dm.plot_confusion_matrix(decision_points=['Stage_0'], savefig=False)

In [ ]:
dm.plot_feature_importance(decision_points=['Stage_0'], savefig=False)

Method __plot_feature_importance__ additionally has two more parameters:
- __drop_outliers__ - whether to remove outliers for quantitative traits
- __clf_results__ - whether to draw the distribution of features by classification results (True) or by source log (False)

In [ ]:
dm.plot_feature_distribution(decision_points=['Stage_0'], drop_outliers=True, clf_results=True, savefig=False)

Such methods as __plot_decision_tree__ and __print_decision_rule__ output the results of the decision mining algorithm as a tree and rules.

Parameters of __plot_decision_tree__:
- __decision_points__ - points for which the decision tree should be drawn
- __max_depth__ - maximum depth of the tree
- __scale__ - scale of the chart
- __savefig__ - whether the image should be saved

In [ ]:
dm.plot_decision_tree(decision_points=['Stage_0'], max_depth=None, scale=1, savefig=False)

Parameters of __print_decision_rule__:
- __decision_points__ - points for which you want to print decisive rules
- __paths__ - paths along which you want to print decisive rules

In [ ]:
dm.print_decision_rule(decision_points=['Stage_0'], paths=['Stage_1'])

## XII. Timing

Calculation of the process duration with pre-cleaning of the outputs using machine learning algorithms.

- __data_holder (SberPM DataHolder)__ - class with the stored data 
- __start_query__ (str) - request specifies the beginning of the new process
- __end_query__ (str) - request that specifies the end of the process
- __query__ (str) - request that points to the beginning of a new process or to the end of the process in this line
- __change_columns__ (List[str]) - list with the names of the columns that can be used to determine that a new process started by changing a value in a column (for example changing process ID or user) 
- __sort_params__ (List[str]) - list of columns names by which the preliminary data sorting will be done

Parameters __query__, __start_query__, __end_query__ can be __"sql"__ or __"pandas"__ type, they both should refer to the data frame as __"df"__, they should return one column: boolean mask or column of 0 and 1.

If a __"sql"__ query is specified, it should look like __"SELECT ... from df"__.

Method __get_chrono()__ will start calculation of process duration and as a result it will output the dictionary(dict) with elements:
- average time of the process in seconds
- number of selected elements
- number of unique processes
- maximum number of unique identifiers calculated in the timeline 


In [ ]:
from sberpm.ml.chronometrage import Chronometrage

In [ ]:
df = pd.read_excel('chrono_data.xlsx', engine='openpyxl')
dh = DataHolder(df, 'process_id', 'event_type', 'data_timestamp')

In [ ]:
example_start_query = """(df['event_type'] == 'Процесс_16961') & (df['event_action'].isin(['Начало']))"""
example_end_query = """(df['event_type'] == 'Процесс_16961') & (df['event_action'].isin(['Конец']))"""

In [ ]:
cr = Chronometrage(dh, 
                   sort_params=['process_id', 'user_id', 'data_timestamp'], 
                   start_query=example_start_query,
                   end_query=example_end_query,
                   change_columns=['process_id', 'user_id'])
res = cr.get_chrono()

In [ ]:
res